## Importer les fichiers et les modules

In [18]:
import pandas as pd
from edc import cvrp
from edc import processing
from edc import utils
import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Méthode 1 : algorithme du voyageur de commerce

In [2]:
df_routes = pd.read_csv("../data/routes_v2.csv")

In [3]:
df_routes["stops_list"] = df_routes.apply(lambda row : row.stops.split(' > '),axis=1)

In [4]:
len(df_routes)

7426

In [5]:
# df_routes['trajectoires_optimisees'] = df_routes.apply(best_traject ,axis=1)
# df_routes.to_csv("..\data\intermediate_data\df_routes_optimisees.csv")

In [6]:
df_routes_optimisees = pd.read_csv("..\data\intermediate_data\df_routes_optimisees.csv")

In [7]:
df_routes_optimisees.trajectoires_optimisees = df_routes_optimisees.trajectoires_optimisees.apply(lambda row : row.replace('[','').replace(']','').split(","))

## Méthode 2 : Vehicule Routing Problem

In [8]:
df_cities = pd.read_csv("../data/cities.csv")
df_orders = pd.read_csv("../data/orders.csv")
df_routes = pd.read_csv("../data/routes.csv")
df_trucks = pd.read_csv("../data/trucks.csv")
df_routes_orders = pd.read_csv("../data/intermediate_data/df_routes_orders.csv")

In [9]:
df_orders.delivered_date = pd.to_datetime( df_orders.delivered_date )

In [10]:
df_1 = pd.merge(df_orders,
                df_cities[["city","lat","lng"]],
                left_on="from_warehouse",
                right_on="city"
                ).rename(columns={'lat':'lat_warehouse','lng':'lng_warehouse'}).drop(columns=["city"])
df_orders_opt = pd.merge(df_1,
                           df_cities[["city","lat","lng"]],
                           left_on="delivery_location",
                           right_on="city").rename(columns={'lat':'lat_delivery','lng':'lng_delivery'}).drop(columns=["city"])
del(df_1)

In [14]:
date_test = datetime.datetime(2021,3,1)
delay_test = 1
warehouse_name_test = 'Cergy'

df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot = processing.processing_opti_rendement(df_orders_opt,warehouse_name_test,date_test,delay_test)
lists_of_cities = cvrp.CVRP(cvrp.create_data_model(df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot),verbose=0)

Objective: 8957 

Total distance of all routes: 8957m
Total load of all routes: 2028.5981000000002


In [21]:
date_test = datetime.datetime(2021,4,2)
delay_test = 1
warehouse_name_test = 'Avignon'

df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot = processing.processing_opti_rendement(df_orders_opt,warehouse_name_test,date_test,delay_test)
lists_of_cities = cvrp.CVRP(cvrp.create_data_model(df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot),verbose=0)
lists_of_cities = [ utils.remove_consecutive_duplicate(list_of_city) for list_of_city in lists_of_cities] 

Objective: 3348 

Total distance of all routes: 3348m
Total load of all routes: 715.58
